Sample of code below.

In [ ]:
import pandas as pd
# Import scikit
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


main_file_path = '../input/train.csv'
# read the data and store data in DataFrame titled melbourne_data
train_data = pd.read_csv(main_file_path) 

# Define Model
price_model = DecisionTreeRegressor()
predictors = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd' ]
# define regressors and dependant variable
y = train_data.SalePrice
X = train_data[predictors]
#Fit Model
price_model.fit(X, y)

data = pd.read_csv(main_file_path)

# print results with head command
print("Making predictions for the following 5 houses:")
print(X.head())
print("The predictions are")
print(price_model.predict(X.head()))

# measure accuracy
#predicted_home_prices = price_model.predict(X)
#mean_absolute_error(y, predicted_home_prices)

# Run a split test on data
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)
# Fit model
price_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = price_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))


# print a summary of the data in Melbourne data
#print(train_data.describe())
#print(train_data.columns)

# check accuracy in different leafs
def get_mae(max_leaf_nodes, predictors_train, predictors_val, targ_train, targ_val):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(predictors_train, targ_train)
    preds_val = model.predict(predictors_val)
    mae = mean_absolute_error(targ_val, preds_val)
    return(mae)

# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))
    
# Introduce a random Forest Regression
forest_model = RandomForestRegressor()
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
print("Random Forest Regression Output for Mean Absolute Error:")
print(mean_absolute_error(val_y, melb_preds))

# Read the test data
test = pd.read_csv('../input/test.csv')
# Treat the test data in the same way as training data. In this case, pull same columns.
test_X = test[predictors]
# Use the model to make predictions
predicted_prices = forest_model.predict(test_X)
# We will look at the predicted prices to ensure we have something sensible.
print(predicted_prices)

# make submission file
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': predicted_prices})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)
